In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import scanpy as sc

from brisc.manuscript_analysis import cell_typing
from brisc.manuscript_analysis import spatial_plots_rabies as spatial

In [ ]:
adata = sc.read_h5ad('C:/brisc/becalia_rabies_barseq/BRAC8498.3e/analysis/adata_q.h5ad')

In [ ]:
atlas_size = 10

# Prepare bin_image for contour plotting
bin_image = spatial.prepare_area_labels(
    xpos=830,
    structures=[
        # Cortical plate layers (Allen CCF acronyms)
        "root",
        "CTX",
        "MB",
        "DG",
        "DG-mo",
        "DG-sg",
        "SCdg",
        "SCdw",
        "SCig",
        "SCiw",
        "SCop",
        "SCsg",
        "SCzo",
        "PAG",
        "MRN",
        "TH",
        "RN",
    ],
    atlas_size=10)

In [ ]:
atlas_size = 10 
chambers = ["chamber_07"]
clusters_not_used = ["Unassigned", "Zero_correlation", "VLMC"]
cortex_exclude = ["fiber_tract", "non_cortical", "TH", "hippocampal"]
qc = dict(best_score=0.3, knn_agree_conf=0.3, raw_gene_counts=2)

layer_tops = {
    "2/3": 116.8406715462,
    "4": 349.9050202564,
    "5": 477.8605504893,
    "6a": 717.1835081307,
    "6b": 909.8772394508,
    "wm": 957.0592130899,
}
blacklist = ["Unassigned", "Oligo", "L6 CT ENT", "L2/3 IT ENT", "VLMC"]
glutamatergic_types = [
    "Car3", "L2/3 IT 1", "L2/3 IT 2", "L2/3 IT RSP",
    "L4 IT", "L4 RSP", "L5 IT", "L5 NP", "L5 PT", "L5/6 IT", "L6 CT", "L6b",
]
gabaergic_types = ["Lamp5", "Pvalb", "Sst", "Vip"]



# ---------------- assemble the full figure ----------------
cm = 1 / 2.54
fontsize_dict = {"title": 7, "label": 8, "tick": 6, "legend": 6}
plt.rcParams.update({
    "axes.titlesize": fontsize_dict["title"],
    "axes.labelsize": fontsize_dict["label"],
    "xtick.labelsize": fontsize_dict["tick"],
    "ytick.labelsize": fontsize_dict["tick"],
    "legend.fontsize": fontsize_dict["legend"],
    "legend.title_fontsize": fontsize_dict["legend"],
})


fig = plt.figure(figsize=(17.4 * cm, 20 * cm), dpi=300, constrained_layout=True)
gs = fig.add_gridspec(nrows=2, ncols=1, height_ratios=[5, 3], hspace=0.1)

clusters, axd = cell_typing.plot_cluster_mosaic(
    fig, gs[0], adata, bin_image, fontsize_dict,
    group_key="custom_leiden", chambers=chambers, clusters_not_used=clusters_not_used,
    cortex_exclude=cortex_exclude, qc=qc, atlas_size=atlas_size, ncols=4,
)

# bottom row: legend + KDE + KDE
subgs_kde = gs[1].subgridspec(nrows=1, ncols=3, width_ratios=[1, 1, 1], wspace=0.35)
ax_leg = fig.add_subplot(subgs_kde[0, 0])
ax_glu = fig.add_subplot(subgs_kde[0, 1])
ax_gaba = fig.add_subplot(subgs_kde[0, 2], sharey=ax_glu)

cell_typing.plot_kde_panels(
    ax_leg, ax_glu, ax_gaba,
    adata=adata, fontsize_dict=fontsize_dict, group_key="custom_leiden",
    layer_tops=layer_tops, blacklist=blacklist,
    glutamatergic_types=glutamatergic_types, gabaergic_types=gabaergic_types,
    x_min=1970, x_max=2260, bw_method=0.1,
)

plt.show()

# fig.savefig("suppfig7_custom_leiden_a4_with_kde.pdf", bbox_inches="tight")